In [1]:
# '''
# Written by Tamas Gabor Csapo <csapot@tmit.bme.hu>
# First version Jan 21, 2019
# Restructured Jan 21, 2020 - for MRI data
# Keras implementation of Csapó T.G., ,,Speaker dependent acoustic-to-articulatory inversion using real-time MRI of the vocal tract'', accepted at Interspeech 2020
# code for inference (MRI video generation)
# '''

# import numpy as np
# import matplotlib.pyplot as plt
# import scipy.io.wavfile as io_wav
# import os
# import os.path
# import glob
# import pickle

# import cv2
# import numpy as np
# from cv2 import VideoWriter, VideoWriter_fourcc

# from subprocess import call, check_output, run

# # import vocoder_LSP_sptk

# from keras.models import model_from_json

# from keras.models import Sequential
# from keras.layers import Dense
# from keras.callbacks import EarlyStopping, CSVLogger, ModelCheckpoint
# from keras.metrics import mean_squared_error

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler, MinMaxScaler

# import subprocess
# from subprocess import run
# import numpy as np
# import scipy.io.wavfile as io_wav

# # do not use all GPU memory
# import tensorflow as tf
# # from keras.backend import set_session
# import tensorflow
# from tensorflow.compat.v1.keras.backend import set_session
# import librosa



# config = tf.compat.v1.ConfigProto()
# # config.gpu_options.per_process_gpu_memory_fraction = 0.3
# config.gpu_options.allow_growth = True
# set_session(tf.compat.v1.Session(config=config))
import numpy as np
import matplotlib.pyplot as plt
import scipy.io.wavfile as io_wav
import os
import glob
import pickle
import cv2
from cv2 import VideoWriter, VideoWriter_fourcc
import librosa
from keras.models import model_from_json
from keras import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, CSVLogger, ModelCheckpoint
from keras.metrics import MeanSquaredError
import tensorflow as tf
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.saving import register_keras_serializable
from tensorflow.keras import backend as K
import subprocess  # Add this import
from subprocess import run

# Register the Sequential class if Keras fails to locate it
register_keras_serializable()(Sequential)

# Limit GPU memory usage in TensorFlow 2.x
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)









# from LipReading with slight modifications
# https://github.com/hassanhub/LipReading/blob/master/codes/data_integration.py
################## VIDEO INPUT ##################
def load_video_3D(path, framesPerSec):

    cap = cv2.VideoCapture(path)
    frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT ))
    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH ))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # make sure that all the videos are the same FPS
    if (np.abs(fps - framesPerSec) > 0.01):
        print('fps:', fps, '(' + path + ')')
        buf = np.empty((frameHeight, frameWidth, frameCount), np.dtype('float32'))

    buf = np.empty((frameHeight, frameWidth, frameCount), np.dtype('float32'))
    fc = 0
    ret = True

    while (fc < frameCount  and ret):
        ret, frame = cap.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame = frame.astype('float32')
        # min-max scaling to [0-1]
        frame = frame-np.amin(frame)
        # make sure not to divide by zero
        if np.amax(frame) != 0:
            frame = frame/np.amax(frame)
        buf[:,:,fc]=frame
        fc += 1
    cap.release()

    return buf

# load vocoder features,
# or calculate, if they are not available
def get_mgc_lsp_coeff(basefilename):
    if os.path.isfile(basefilename + '.mgclsp'):
        mgc_lsp_coeff = np.fromfile(basefilename + '.mgclsp', dtype=np.float32).reshape(-1, order + 1)
        lf0 = np.fromfile(basefilename + '.lf0', dtype=np.float32)
    else:
        (mgc_lsp_coeff, lf0) = vocoder_LSP_sptk.encode(basefilename, samplingFrequency, frameLength, frameShift, order, alpha, stage)
    return (mgc_lsp_coeff, lf0)

# convert an array of values into a dataset matrix
# code with modifications from
# https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/
def create_dataset_img(data_in_X, data_in_Y, look_back=1):
    (dim1_X, dim2_X, dim3_X, dim4_X) = data_in_X.shape
    (dim1_Y, dim2_Y) = data_in_Y.shape
    data_out_X = np.empty((dim1_X - look_back - 1, look_back, dim2_X, dim3_X, dim4_X))
    data_out_Y = np.empty((dim1_Y - look_back - 1, dim2_Y))

    for i in range(dim1_X - look_back - 1):
        for j in range(look_back):
            data_out_X[i, j] = data_in_X[i + j]
        data_out_Y[i] = data_in_Y[i + j]
    return data_out_X, data_out_Y

# convert an array of values into a dataset matrix
# code with modifications from
# https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/
def create_dataset_img_inverse(data_in_X, data_in_Y, look_back=1):
    (dim1_X, dim2_X) = data_in_X.shape
    (dim1_Y, dim2_Y, dim3_Y, dim4_Y) = data_in_Y.shape
    data_out_X = np.empty((dim1_X - look_back - 1, look_back, dim2_X))
    data_out_Y = np.empty((dim1_Y - look_back - 1, dim2_Y, dim3_Y, dim4_Y))

    for i in range(dim1_X - look_back - 1):
        for j in range(look_back):
            data_out_X[i, j] = data_in_X[i + j]
        data_out_Y[i] = data_in_Y[i + j]
    return data_out_X, data_out_Y

# mri2vid converts raw MRI data to .mp4 video
def mri2vid(mri_data, dir_file, filename_no_ext, n_width, n_height, FramesPerSec):

    print(filename_no_ext + ' - MRI video started')

    output_file_no_ext = dir_file + filename_no_ext
    n_frames = len(mri_data)

    # compressed
    # fourcc = VideoWriter_fourcc(*'MP4V')

    # uncompressed 8-bit
    fourcc = VideoWriter_fourcc(*'Y800')
    video = VideoWriter(output_file_no_ext + '.avi', fourcc, float(FramesPerSec), (n_width, n_height), 0)

    for n in range(n_frames):
        frame = np.uint8(255 * mri_data[n]).reshape(n_width, n_height, 1)

        video.write(frame)
        print('frame ', n, ' done', end='\r')

    video.release()

    print(filename_no_ext + ' - MRI video finished')

def mrividwav2demo(dir_mri, file_mri, dir_wav, file_wav):
    # "-codec copy " + \
    command = "ffmpeg " + \
           "-y " + \
           "-i " + dir_mri + file_mri + " " + \
           "-i " + dir_wav + file_wav + " " + \
           "-shortest " + \
           "-acodec copy -vcodec copy " + \
            dir_mri + file_mri[:-4] + "_with_audio.avi"
           # "-c:v h264 -crf 20 -c:a aac -strict -2 " + \
           # "-filter:v \"crop=820:496:215:48\" " + \

    print(command)
    run(command, shell=True)



# speakerlist = ['F1']
# model_path = '/content/drive/MyDrive/GAN_based_models/speech2mri/models/Models_for_Text2MRI/'
# # output_path = '/content/drive/MyDrive/GAN_based_models/speech2mri/Video_generation_code_for_NCC_paper/generated_image_sequence/'
# output_path = '/content/drive/MyDrive/GAN_based_models/speech2mri/Video_generation_code_for_NCC_paper/generated_image_sequence_original_audio/'


# # Input_audio_path = '/content/drive/MyDrive/GAN_based_models/speech2mri/Video_generation_code_for_NCC_paper/audio_files_for_input/'
# Input_audio_path = '/content/drive/MyDrive/GAN_based_models/speech2mri/Video_generation_code_for_NCC_paper/original_audio_files/'



speakerlist = ['F1', 'F2', 'M2', 'M3']
model_path = '/content/drive/MyDrive/GAN_based_models/speech2mri/models_speech2mri_2024/'
# output_path = '/content/drive/MyDrive/GAN_based_models/speech2mri/testing_models/'
output_path = '/content/drive/MyDrive/GAN_based_models/speech2mri/Video_generation_code_for_NCC_paper/temp_image_sequence/'

# Input_audio_path = '/content/drive/MyDrive/GAN_based_models/wav_files/'
# Input_audio_path = '/content/drive/MyDrive/GAN_based_models/speech2mri/Video_generation_code_for_NCC_paper/original_audio_files/'
Input_audio_path = '/content/drive/MyDrive/GAN_based_models/speech2mri/Video_generation_code_for_NCC_paper/test_audio_files/'


speakerInd = 0
# for speaker in ['f1']: # ['f1', 'f2', 'm1', 'm2']:
for speaker in speakerlist:
    # TODO: modify this according to your data path
    # dir_mri = '/content/drive/MyDrive/backup_PhD/database/MRI_USC/data/' + speaker + '/avi/'
    # dir_mri_test = '/content/drive/MyDrive/GAN_based_models/speech2mri/Video_generation_code_for_NCC_paper/temp_image_sequence/' + speaker + '/'
    dir_mri_test = '/content/drive/MyDrive/GAN_based_models/speech2mri/Video_generation_code_for_NCC_paper/test_video_generated/LSTM_10_outputs/' + speaker + '/'


    if not os.path.exists(dir_mri_test):
        os.makedirs(dir_mri_test)

    # Parameters of vocoder
    samplingFrequency = 20000
    frameLength = 1024 #
    frameShift = 863 # 43.14 ms at 20000 Hz sampling, correspondong to 23.18 fps (MRI video)
    order = 24
    alpha = 0.42
    stage = 3
    n_mgc = order + 1

    # context window of LSTM
    n_sequence = 10

    # properties of MRI videos
    framesPerSec = 23.18
    n_width = 68
    n_height = 68

    # modelbasenames = [ 'SPEECH2MRI_LSTM_baseline_Text2MRI_M2_2023-04-18_10-55-58',
    #           'SPEECH2MRI_LSTM_baseline_Text2MRI_M3_2023-04-18_12-28-25',
    #           'SPEECH2MRI_LSTM_baseline_Text2MRI_F1_2023-04-18_13-44-16',
    #           'SPEECH2MRI_LSTM_baseline_F2_2024-10-04_10-04-46']

    modelbasenames = [ 'SPEECH2MRI_LSTM_baseline_F1_2024-10-04_09-57-40',
                      'SPEECH2MRI_LSTM_baseline_F2_2024-10-04_10-04-46',
                       'SPEECH2MRI_LSTM_baseline_M2_2024-10-04_09-33-21',
                       'SPEECH2MRI_LSTM_baseline_M3_2024-10-04_09-52-15']

    # modelbasenames = [ 'SPEECH2MRI_Transformer_baseline_F1_2024-10-27_07-42-48',
    #                   'SPEECH2MRI_Transformer_baseline_F2_2024-10-27_07-46-43',
    #                    'SPEECH2MRI_Transformer_baseline_M2_2024-10-27_07-36-15',
    #                    'SPEECH2MRI_Transformer_baseline_M3_2024-10-27_07-40-11']


    # DNN_types = ['FC-DNN_baseline', 'CNN', 'LSTM']
    #DNN_types = ['FC-DNN_baseline']
    # DNN_types = ['LSTM-CNN']
    DNN_types = ['LSTM']
    # DNN_types = ['Transformer']
    # basefilenames_mri_test = ['usctimit_mri_' + speaker.lower() + '_146_150', 'usctimit_mri_' + speaker.lower() + '_441_445']
    basefilenames_mri_test = glob.glob(Input_audio_path + speaker +'/*.wav')

    for DNN_type in DNN_types:
        # e.g. MRI2SPEECH_CNN_f1_2020-01-16_10-36-35
        # csv_files = glob.glob('/content/drive/MyDrive/GAN_based_models/speech2mri/models/SPEECH2MRI_' + DNN_type + '_baseline_Text2MRI_' + speaker + '*.csv')
        csv_files = glob.glob(model_path + modelbasenames[speakerInd] +'*.csv')
        model_name = csv_files[-1][:-4]

        speakerInd = speakerInd + 1
        # load model
        print('loading model', model_name)
        with open(model_name + '_model.json', "r") as json_file:
            loaded_model_json = json_file.read()
        # model = model_from_json(loaded_model_json)

        model = model_from_json(loaded_model_json, custom_objects={'Sequential': Sequential})
        model.load_weights(model_name + '_weights.keras')
        print("Loaded model from disk")
        # load weights into new model
        # model.load_weights(model_name + '_weights.h5')
        # load scalers
        mgc_scalers = pickle.load(open(model_name + '_mgc_scalers.sav', 'rb'))

        for basefilename in basefilenames_mri_test:
            print('Predicting output for: ', basefilename)

            # load data for sentence
            # mri_data = load_video_3D(dir_mri + basefilename + '.avi', framesPerSec)
            # mri_len = mri_data.shape[2]
            # mri_test = np.empty((mri_len, n_width, n_height))
            # (mgc_lsp_coeff, lf0) = get_mgc_lsp_coeff(dir_mri + basefilename)
            # dir_mri_wav_only = dir_mri.replace('/avi/','/wav/')
            # dir_mri_wav = dir_mri_wav_only + basefilename+'.wav'
            dir_mri_wav = basefilename
            basefilename_name_only = basefilename.split('/')
            basefilename_name_only = basefilename_name_only[-1]
            basefilename_name_only = basefilename_name_only.replace('.wav', '')

            print(basefilename_name_only)
            x, sr = librosa.load(dir_mri_wav, sr = samplingFrequency)
            n_fft = frameLength   # window length: 0.02 s
            hop_length = frameShift  #
            mgc_lsp_coeff = librosa.feature.mfcc(y=x, sr=sr, n_mfcc=25, hop_length=hop_length, n_fft=n_fft)
            mgc_lsp_coeff = mgc_lsp_coeff.transpose()



            # for i in range(mri_len):
            #     mri_test[i] = mri_data[:, :, i] # original, 68x68

            # transform of input parameters
            for i in range(n_mgc):
                mgc_lsp_coeff[:, i] = mgc_scalers[i].transform(mgc_lsp_coeff[:, i].reshape(-1, 1)).ravel()

            # reshape for LSTM
            if DNN_type == 'LSTM' or DNN_type == 'LSTM-CNN' or DNN_type == 'Transformer':
                mgc_len = len(mgc_lsp_coeff)
                mri0 = np.empty((mgc_len, n_width, n_height, 1))

                mgc_test0, mri0 = create_dataset_img_inverse(mgc_lsp_coeff, mri0, look_back = n_sequence)
                mri0 = mri0.reshape(-1, n_width * n_height)
                mgc_test = np.empty((mgc_len, n_sequence, n_mgc))

                # # add first n_sequence values
                # for i in range(mgc_len - 2):
                #     if i < n_sequence - 0:
                #         mgc_test[i] = mgc_test0[0]
                #     else:
                #         mgc_test[i] = mgc_test0[i - n_sequence + 1]

                # mgc_lsp_coeff = mgc_test

            mgc_lsp_coeff = mgc_test0
            # predict MR image sequence using the trained model
            mri_predicted = model.predict(mgc_lsp_coeff)
            print('Prediction done')
            # clip extreme values
            mri_predicted = np.clip(mri_predicted, 0, 1)

            print(mri_predicted.shape)

            y_pred = mri_predicted
            y_true = mri0

            # # Calculating the error
            # FrameErr = np.zeros((y_true.shape[0],1))
            # for i in range(y_true.shape[0]):
            #   t1 = y_pred[i,:]
            #   t2 = y_true[i,:]
            #   terr =np.mean((np.square(t1-t2)))
            #   FrameErr[i]=terr

            # MSErr = np.mean(FrameErr)
            # print(MSErr)

            # MSErr_fn = mean_squared_error(y_pred, y_true)
            # print(np.mean(MSErr_fn))


            # save image sequence to video (without audio)
            mri2vid(mri_predicted, dir_mri_test, basefilename_name_only + '_' + DNN_type, n_width, n_height, framesPerSec)

            dir_mri_wav_only = Input_audio_path + speaker +'/'
            # put together video and audio
            mrividwav2demo(dir_mri_test, basefilename_name_only + '_' + DNN_type + '.avi', \
                dir_mri_wav_only, basefilename_name_only + '.wav')




loading model /content/drive/MyDrive/GAN_based_models/speech2mri/models_speech2mri_2024/SPEECH2MRI_LSTM_baseline_F1_2024-10-04_09-57-40
Loaded model from disk
Predicting output for:  /content/drive/MyDrive/GAN_based_models/speech2mri/Video_generation_code_for_NCC_paper/test_audio_files/F1/usctimit_mri_f1_441_445.wav
usctimit_mri_f1_441_445
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step
Prediction done
(591, 4624)
usctimit_mri_f1_441_445_LSTM - MRI video started
usctimit_mri_f1_441_445_LSTM - MRI video finished
ffmpeg -y -i /content/drive/MyDrive/GAN_based_models/speech2mri/Video_generation_code_for_NCC_paper/test_video_generated/LSTM_10_outputs/F1/usctimit_mri_f1_441_445_LSTM.avi -i /content/drive/MyDrive/GAN_based_models/speech2mri/Video_generation_code_for_NCC_paper/test_audio_files/F1/usctimit_mri_f1_441_445.wav -shortest -acodec copy -vcodec copy /content/drive/MyDrive/GAN_based_models/speech2mri/Video_generation_code_for_NCC_paper/test_video_generated/LSTM_10_outputs/F1/usctimit_mri_f1_4

In [ ]:

import subprocess
from subprocess import run
mrividwav2demo(dir_mri_test, basefilename_name_only + '_' + DNN_type + '.avi', \
                dir_mri_wav_only, basefilename_name_only + '.wav')


ffmpeg -y -i /content/drive/MyDrive/GAN_based_models/speech2mri/Video_generation_code_for_NCC_paper/temp_image_sequence/F1/usctimit_mri_f1_441_445_LSTM.avi -i /content/drive/MyDrive/GAN_based_models/speech2mri/Video_generation_code_for_NCC_paper/original_audio_files/F1/usctimit_mri_f1_441_445.wav -shortest -acodec copy -vcodec copy /content/drive/MyDrive/GAN_based_models/speech2mri/Video_generation_code_for_NCC_paper/temp_image_sequence/F1/usctimit_mri_f1_441_445_LSTM_with_audio.avi


In [5]:
import os
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim

def calculate_video_ssim(video_path1, video_path2):
    cap1 = cv2.VideoCapture(video_path1)
    cap2 = cv2.VideoCapture(video_path2)
    ssim_values = []

    while True:
        ret1, frame1 = cap1.read()
        ret2, frame2 = cap2.read()

        if not ret1 or not ret2:
            break

        gray_frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        gray_frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

        ssim_value, _ = ssim(gray_frame1, gray_frame2, full=True)
        ssim_values.append(ssim_value)

    cap1.release()
    cap2.release()
    return np.mean(ssim_values), np.std(ssim_values)

# Define paths to the main directories for folder1 and folder2
main_folder2 = '/content/drive/MyDrive/backup_PhD/database/MRI_USC/data/'
main_folder1 = '/content/drive/MyDrive/GAN_based_models/speech2mri/Video_generation_code_for_NCC_paper/test_video_generated/LSTM_10_outputs/'

# Dictionary to store SSIM results for each speaker
overall_ssim_results = {}

# Loop over each speaker directory in folder1
for speaker_folder in os.listdir(main_folder1):
    speaker_path1 = os.path.join(main_folder1, speaker_folder)
    speaker_path2 = os.path.join(main_folder2, speaker_folder, 'avi')

    # Skip if it's not a directory or if there's no matching speaker folder in folder2
    if not os.path.isdir(speaker_path1) or not os.path.exists(speaker_path2):
        continue

    ssim_results = []

    # Loop over each file in the speaker's folder
    for file_name in os.listdir(speaker_path1):
        if not file_name.endswith("Transformer.avi") and (not file_name.endswith("LSTM.avi")):
            continue

        video_path1 = os.path.join(speaker_path1, file_name)
        dest_file_name  = file_name.replace('_Transformer', '')
        video_path2 = os.path.join(speaker_path2, dest_file_name)

        # Check if the file exists in both folders
        if os.path.exists(video_path2):
            mean_ssim, std_ssim = calculate_video_ssim(video_path1, video_path2)
            ssim_results.append(mean_ssim)
            print(f"Speaker: {speaker_folder} - File: {file_name} - Mean SSIM: {mean_ssim:.4f}, Std SSIM: {std_ssim:.4f}")
        else:
            print(f"File {file_name} not found in both folders for Speaker {speaker_folder}.")

    # Calculate mean and standard deviation of SSIM for each speaker
    overall_ssim_results[speaker_folder] = {
        'mean_ssim': np.mean(ssim_results),
        'std_ssim': np.std(ssim_results)
    }

# Print overall results for each speaker
print("\nOverall SSIM Results for Each Speaker:")
for speaker, results in overall_ssim_results.items():
    print(f"{speaker} - Mean SSIM: {results['mean_ssim']:.4f}, Std SSIM: {results['std_ssim']:.4f}")


File usctimit_mri_f1_441_445_LSTM.avi not found in both folders for Speaker F1.
File usctimit_mri_f1_146_150_LSTM.avi not found in both folders for Speaker F1.
File usctimit_mri_f1_261_265_LSTM.avi not found in both folders for Speaker F1.
File usctimit_mri_f1_121_125_LSTM.avi not found in both folders for Speaker F1.
File usctimit_mri_f1_316_320_LSTM.avi not found in both folders for Speaker F1.
File usctimit_mri_f1_411_415_LSTM.avi not found in both folders for Speaker F1.
File usctimit_mri_f1_101_105_LSTM.avi not found in both folders for Speaker F1.
File usctimit_mri_f1_061_065_LSTM.avi not found in both folders for Speaker F1.
File usctimit_mri_f1_091_095_LSTM.avi not found in both folders for Speaker F1.
File usctimit_mri_f1_396_400_LSTM.avi not found in both folders for Speaker F1.
File usctimit_mri_f2_441_445_LSTM.avi not found in both folders for Speaker F2.
File usctimit_mri_f2_146_150_LSTM.avi not found in both folders for Speaker F2.
File usctimit_mri_f2_261_265_LSTM.avi no

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [7]:
import os
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim

def calculate_video_ssim(video_path1, video_path2):
    cap1 = cv2.VideoCapture(video_path1)
    cap2 = cv2.VideoCapture(video_path2)
    ssim_values = []

    while True:
        ret1, frame1 = cap1.read()
        ret2, frame2 = cap2.read()

        if not ret1 or not ret2:
            break

        gray_frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        gray_frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

        ssim_value, _ = ssim(gray_frame1, gray_frame2, full=True)
        ssim_values.append(ssim_value)

    cap1.release()
    cap2.release()
    return np.mean(ssim_values), np.std(ssim_values)

# Define paths to the main directories for folder1 and folder2
main_folder2 = '/content/drive/MyDrive/backup_PhD/database/MRI_USC/data/'
main_folder1 = '/content/drive/MyDrive/GAN_based_models/speech2mri/Video_generation_code_for_NCC_paper/test_video_generated/LSTM_10_outputs/'

# Dictionary to store SSIM results for each speaker
overall_ssim_results = {}

# Loop over each speaker directory in folder1
for speaker_folder in os.listdir(main_folder1):
    speaker_path1 = os.path.join(main_folder1, speaker_folder)
    speaker_path2 = os.path.join(main_folder2, speaker_folder, 'avi')

    # Skip if it's not a directory or if there's no matching speaker folder in folder2
    if not os.path.isdir(speaker_path1) or not os.path.exists(speaker_path2):
        continue

    ssim_results = []

    # Loop over each file in the speaker's folder
    for file_name in os.listdir(speaker_path1):
        if not file_name.endswith("LSTM.avi"):
            continue

        video_path1 = os.path.join(speaker_path1, file_name)
        dest_file_name  = file_name.replace('_LSTM', '')
        video_path2 = os.path.join(speaker_path2, dest_file_name)

        # Check if the file exists in both folders
        if os.path.exists(video_path2):
            mean_ssim, std_ssim = calculate_video_ssim(video_path1, video_path2)
            ssim_results.append(mean_ssim)
            print(f"Speaker: {speaker_folder} - File: {file_name} - Mean SSIM: {mean_ssim:.4f}, Std SSIM: {std_ssim:.4f}")
        else:
            print(video_path2)
            print(f"File {file_name} not found in both folders for Speaker {speaker_folder}.")

    # Calculate mean and standard deviation of SSIM for each speaker
    overall_ssim_results[speaker_folder] = {
        'mean_ssim': np.mean(ssim_results),
        'std_ssim': np.std(ssim_results)
    }

# Print overall results for each speaker
print("\nOverall SSIM Results for Each Speaker:")
for speaker, results in overall_ssim_results.items():
    print(f"{speaker} - Mean SSIM: {results['mean_ssim']:.4f}, Std SSIM: {results['std_ssim']:.4f}")


Speaker: F1 - File: usctimit_mri_f1_441_445_LSTM.avi - Mean SSIM: 0.7689, Std SSIM: 0.0738
Speaker: F1 - File: usctimit_mri_f1_146_150_LSTM.avi - Mean SSIM: 0.7995, Std SSIM: 0.0564
Speaker: F1 - File: usctimit_mri_f1_261_265_LSTM.avi - Mean SSIM: 0.8193, Std SSIM: 0.0616
Speaker: F1 - File: usctimit_mri_f1_121_125_LSTM.avi - Mean SSIM: 0.8155, Std SSIM: 0.0495
Speaker: F1 - File: usctimit_mri_f1_316_320_LSTM.avi - Mean SSIM: 0.7812, Std SSIM: 0.0494
Speaker: F1 - File: usctimit_mri_f1_411_415_LSTM.avi - Mean SSIM: 0.7566, Std SSIM: 0.0862
Speaker: F1 - File: usctimit_mri_f1_101_105_LSTM.avi - Mean SSIM: 0.8135, Std SSIM: 0.0626
Speaker: F1 - File: usctimit_mri_f1_061_065_LSTM.avi - Mean SSIM: 0.8204, Std SSIM: 0.0590
Speaker: F1 - File: usctimit_mri_f1_091_095_LSTM.avi - Mean SSIM: 0.8201, Std SSIM: 0.0612
Speaker: F1 - File: usctimit_mri_f1_396_400_LSTM.avi - Mean SSIM: 0.8053, Std SSIM: 0.0625
Speaker: F2 - File: usctimit_mri_f2_441_445_LSTM.avi - Mean SSIM: 0.7067, Std SSIM: 0.0644